In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as rd
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I


In [2]:
m = ModelIntv()
n_collect = 200
seed = 11667
rd.seed(seed)

In [3]:
out_path = f'out/dy_ladd'
pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
sel = rd.choice(list(range(len(pars_post))), n_collect)
sel.sort()
pars_post = [pars_post[i] for i in sel]

cov_bg = json.load(open(f'{out_path}/CoverageBg.json', 'r'))

In [4]:
intv00 = {}
intv01 = {'FullACF': cov_bg['MDU']['M']}
intv10 = {'AltACF': cov_bg['D2D']['M']}
intv11 = {'FullACF': cov_bg['MDU']['M'], 'AltACF': cov_bg['D2D']['M']}

intv2x = {k: dict(v) for k, v in intv11.items()}
for v in intv2x.values():
    v['Coverage'] *= 2
intv4x = {k: dict(v) for k, v in intv11.items()}
for v in intv4x.values():
    v['Coverage'] *= 4

In [5]:
y1p = [m.find_baseline(p, 2022) for i, p in enumerate(pars_post)]

In [6]:
results = list()

for i, (y1, p) in enumerate(y1p):
    _, (_, ms00), _ = m.simulate_onward(y1, p, intv=intv00, polished=True)
    _, (_, ms01), _ = m.simulate_onward(y1, p, intv=intv01, polished=True)
    _, (_, ms10), _ = m.simulate_onward(y1, p, intv=intv10, polished=True)
    _, (_, ms11), _ = m.simulate_onward(y1, p, intv=intv11, polished=True)
    
    results.append({
        'Key': i,
        'Avt_MDU': 1 - ms01['Inc'] / ms00['Inc'],
        'Avt_D2D': 1 - ms10['Inc'] / ms00['Inc'],
        'Avt_MDU+D2D': 1 - ms11['Inc'] / ms00['Inc'],
    })
    
results = pd.DataFrame(results)

In [7]:
results.describe()

,Key,Avt_MDU,Avt_D2D,Avt_MDU+D2D
count,200.000000,200.000000,200.000000,200.000000
mean,99.500000,0.008757,0.003743,0.012338
std,57.879185,0.002639,0.002200,0.004665
min,0.000000,0.004591,0.000919,0.005520
25%,49.750000,0.006723,0.002002,0.008669
50%,99.500000,0.008231,0.003136,0.011320
75%,149.250000,0.010489,0.004943,0.015206
max,199.000000,0.015602,0.010503,0.025217


In [8]:
results.to_csv(f'{out_path}/Sim_BgACF_IncrementalAverted.csv')

In [11]:
mss = list()
mss_stats = list()

for i, (y1, p) in enumerate(y1p):
    _, ms0, _ = m.simulate_onward(y1, p, intv=intv00, polished=True)
    _, ms1, _ = m.simulate_onward(y1, p, intv=intv11, polished=True)
    _, ms2, _ = m.simulate_onward(y1, p, intv=intv2x, polished=True)
    _, ms4, _ = m.simulate_onward(y1, p, intv=intv4x, polished=True)

    mss.append(pd.concat([
        ms0[0].reset_index().assign(Key=i, Scale='Baseline'),
        ms1[0].reset_index().assign(Key=i, Scale='1x'),
        ms2[0].reset_index().assign(Key=i, Scale='2x'),
        ms4[0].reset_index().assign(Key=i, Scale='4x')
    ]))
    
    ms0[1].update({'Key': i, 'Scale': 'Baseline'})
    ms1[1].update({'Key': i, 'Scale': '1x'})
    ms2[1].update({'Key': i, 'Scale': '2x'})
    ms4[1].update({'Key': i, 'Scale': '4x'})
    mss_stats += [ms0[1], ms1[1], ms2[1], ms4[1]]
    

In [12]:
mss = pd.concat(mss)
mss_stats = pd.DataFrame(mss_stats)

In [13]:
mss.to_csv(f'{out_path}/Sim_BgACF_ScaleUp.csv')
mss_stats.to_csv(f'{out_path}/Sim_BgACF_ScaleUp_Stats.csv')